# 메모리

대화 내용을 기억하는 것 - *요즘은 랭그래프에 다 흡수됨*

- LLM은 기본적으로 대화 내용을 기억하지 않음(stateless) - input 1번에 output 1번

- 계속 이전 대화내용을 프롬프트에 넣어줘야함

1. Short-Term Memory(단기기억)

    한 대화 세션 안에 대한 기억

2. Long-Term Memory(장기기억)

    전체 세션에서 추출한 중요한 정보 - GPT에서 메모리 업데이트됨 같은것

## 메모리는 어디에 저장해야 하는가?

1. 디스크

    속도가 느림 / 저장하기 좋다(안정성이 높음)

    기본적인 모든 데이터 저장

2. 램

    엄청 빠름 / 컴퓨터가 꺼졌다 켜지면 사라짐(휘발성)

    자주 확인하는 데이터, 지금 사용하는 데이터들은 램에 저장 => **캐싱**

## Conversation Buffer Memory

메시지를 저장한 뒤 변수에서 추출

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)  # 사전적 의미: 랜덤성, 확률이 낮은 단어도 선택 가능함
# temperature 높음: 설명을 하기 위해 다양한 언어를 사용, 지루하고 현학적임 - 창의적, 예술적이거나 생각이 필요한 경우는 temp가 높아야함
# temperature 낮음: 팩트 위주, 딱 사실만 반영 - 논문 참조, 정보 확인 등 정확성이 중요할 경우 temp가 낮은게 맞음

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = ChatPromptTemplate(
    [
        ('system', '넌 정확한 정보를 전달하는 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 여기에 기존 채팅의 모든 내역을 주입함 - 내역을 넣을 공간을 미리 채워두는것
        ('human', '{input}')
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리는 {}에 저장, 기존에 대화내용이 있다면 불러오기
memory.load_memory_variables({})
pass

In [3]:
from operator import itemgetter
# chat_history에 load_memory_var 결과를 저장 후 키를 추출
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | 
    itemgetter('chat_history')
)

In [14]:
chain = runnable | prompt | llm

my_message = '안녕'
res = chain.invoke({'input': my_message})

memory.save_context(
    {'human': my_message},
    {'ai': res.content}    
)
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

## 메모리 구동 방식
기본적으로 대화 내용은 전부 프롬프트에 집어넣음

대화가 길어지면 길어질수록, 소비하는 토큰 숫자가 늘어난다 + 뒤로 갈수록 성능도 낮아짐

- 대화 요약 생성 후 요약을 저장(Conversation Summary Memory)
- 최근 K개의 상호작용만 사용함(Conversation Buffer Window Memory)
- 토큰 길이를 이용해 대화내용을 플러시할 시기를 결정(Conversation Token Buffer Memory)
- ...

이젠 볼 일이 없다. 전부 랭그래프로 기능이 넘어감.

In [30]:
# 실제 gpt랑 대화하는 식으로 만들어 보기
from dotenv import load_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import time

load_dotenv()

input_msg = ''

# 메모리
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | 
    itemgetter('chat_history')
)

# 프롬프트
prompt = ChatPromptTemplate(
    [
        ('system', '항상 존댓말로 대답해'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}')
    ]
)

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=1)

# Chain
chain = runnable | prompt | llm | StrOutputParser()

# 정지 신호 설정
while input_msg not in ('멈춰', '그만', '정지'):
    input_msg = input()
    # 대화 로직 + 대화내역 출력, 정지신호 시 대화 종료
    print('나:')
    print(input_msg)
    print('AI:')
    
    # 대답 생성
    ai_response = ''
    for token in chain.stream({'input': input_msg}):
        print(token, end='', flush=True)
        ai_response += token
    
    # 메모리에 응답 저장
    memory.save_context(
        {'human': input_msg},
        {'ai': ai_response}    
    )
    
    print('\n---')
    time.sleep(0.5)
    
memory.clear()

나:
안녕? 너가 메모리를 기억하는지 실험 중이야. 뭔가 기억하고 싶은 게 있니?
AI:
안녕하세요! 저는 지금 현재 대화 세션 내에서만 정보를 기억할 수 있어서, 이전 만남이나 내용을 계속 기억하지는 못합니다. 그렇지만, 만약 기억했으면 하는 중요한 내용이 있다면 말씀해 주시면 계속 이어서 도움을 드릴 수 있습니다. 어떤 내용을 기억했으면 좋겠나요?
---
나:
너한테 물어보잖아. 뭔가 기억하고 싶은게 있냐고
AI:
네, 질문해 주셔서 감사합니다. 저는 인공지능이기 때문에 개인적인 기억이나 바람이 있는 것은 아니지만, 만약 제가 기억한다면, 사용자님과의 대화에서 유용하거나 의미 있다고 느낄 만한 중요한 정보들을 기억하고 싶어요. 예를 들어, 사용자님이 특별히 관심 있는 분야나 필요한 정보 등을 기억한다면 더 잘 도와드릴 수 있을 것 같습니다. 혹시 어떠한 정보나 주제를 기억했으면 좋겠다고 생각하시는 게 있으신가요?
---
나:
그럼 내가 무채색을 좋아한다는 사실을 기억해봐
AI:
알겠습니다. 사용자님께서 무채색을 좋아하신다는 사실을 기억하겠습니다. 앞으로 대화할 때 참고할게요. 혹시 더 나누고 싶은 정보가 있으시면 언제든 말씀해 주세요.
---
나:
내가 어떤 색을 좋아한다 그랬지?
AI:
네, 사용자님께서 무채색을 좋아하신다고 말씀하셨죠. 앞으로 대화에서 그 점을 참고하겠습니다. 더 궁금하신 점이나 나누고 싶은 이야기가 있으시면 언제든 말씀해 주세요.
---
나:
그만
AI:
알겠습니다. 언제든 필요하시면 말씀해 주세요. 좋은 하루 되시길 바랍니다.
---
